In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
from pprint import pprint
from tqdm.notebook import tqdm

In [2]:
df_sent = pd.read_csv("./data/extracted_sentences.csv", header=None)
pprint(df_sent.iloc[0][0])
print()
pprint(df_sent.iloc[0][1])
# df

temp = df_sent[0].str.len()
sum(temp)


('Pri tem opozarjajo na zakon o kazenskem postopku, ki vsem državnim organom, '
 'tudi Komisiji za preprečevanje korupcije, nalaga, da morajo prijaviti '
 'kazniva dejanja, za katera se storilec preganja po uradni dolžnosti, če so o '
 'njih obveščeni. »Komisija ni obvestila policije o posedovanju dokumentacije, '
 'ki po njihovih izjavah kaže na sum korupcije in kazniva dejanja, temveč smo '
 'od njih dokumentacijo zahtevali sami,« so še navedli na GPU. ')

('Opozarjajo na Zakon o kazenskem postopku, ki od vseh državnih organov, '
 'vključno s Komisijo za preprečevanje korupcije, zahteva, da prijavijo '
 'kazniva dejanja, za katera je storilec preganjan po uradni dolžnosti, če so '
 'o njih obveščeni. "Komisija ni obvestila policije o posedovanju dokumentov, '
 'ki po njihovih izjavah kažejo na sum korupcije in kaznivih dejanj, ampak smo '
 'dokumentacijo od njih zahtevali sami ", so še navedli na GPU.')


2413940

In [3]:
df_para = pd.read_csv("./data/extracted_paragraphs.csv", header=None)
pprint(df_para.iloc[0][0])
print()
pprint(df_para.iloc[0][1])
temp = df_para[0].str.len()
sum(temp)

('Pri tem opozarjajo na zakon o kazenskem postopku, ki vsem državnim organom, '
 'tudi Komisiji za preprečevanje korupcije, nalaga, da morajo prijaviti '
 'kazniva dejanja, za katera se storilec preganja po uradni dolžnosti, če so o '
 'njih obveščeni.\n'
 '»Komisija ni obvestila policije o posedovanju dokumentacije, ki po njihovih '
 'izjavah kaže na sum korupcije in kazniva dejanja, temveč smo od njih '
 'dokumentacijo zahtevali sami,« so še navedli na GPU.\n')

('Pri tem opozarjajo na Zakon o kazenskem postopku, ki od vseh državnih '
 'organov, vključno s Komisijo za preprečevanje korupcije, zahteva, da '
 'prijavijo kazniva dejanja, za katera je storilec preganjan po uradni '
 'dolžnosti, če so o njih obveščeni. "Komisija ni obvestila policije o '
 'posedovanju dokumentacije, ki po njihovih navedbah kaže na sum korupcije in '
 'kaznivih dejanj, ampak smo od njih zahtevali dokumentacijo," so še navedli '
 'na GPU.')


291698

In [4]:
print(f"df_sent: {df_sent.size}")
print(f"df_para: {df_para.size}")

df_sent: 22390
df_para: 1228


In [5]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self,data_path):
        super(MyDataSet).__init__(),
        self.df = pd.read_csv(data_path, header=None)

    def __len__(self):
        return self.df.size
    
    def __getitem__(self,indx):
        return {"input":self.df.iloc[indx][0].strip(),"output":self.df.iloc[indx][1].strip()}

device = "cuda"
lr = 0.001
epochs = 1
batch_size = 128

dataset = MyDataSet("./data/extracted_sentences.csv")
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)


model = transformers.T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
tokenizer = transformers.T5Tokenizer.from_pretrained('t5-small')

optimizer = torch.optim.Adam(params=model.parameters(),lr=lr)

for epoch in range(epochs):
    with tqdm(total=len(dataset), unit="batch") as progress:
        for batch in dataloader:
            inputs = batch["input"]
            outputs = batch["output"]

            input_tokens = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt').input_ids.to(device)
            output_tokens = tokenizer(outputs, padding=True, truncation=True, return_tensors='pt').input_ids.to(device)

            outputs = model(input_ids=input_tokens, labels=output_tokens)
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            optimizer.zero_grad()

            progress.update(len(inputs))
            progress.set_postfix(epoch=epoch, loss=loss.item())


  0%|          | 0/22390 [00:00<?, ?batch/s]

In [ ]:
# train
trainDataset = dataset_module.SynteticTeraseDataset(img_shape,"train")
trainDataLoader = torch.utils.data.DataLoader(dataset=trainDataset, batch_size=batch_size, num_workers=6, prefetch_factor=2, pin_memory=True)
trainIterator = iter(trainDataLoader)
# test
testDataset = dataset_module.SynteticTeraseDataset(img_shape,"test")
testDataLoader = torch.utils.data.DataLoader(dataset=testDataset, batch_size=batch_size, num_workers=1, prefetch_factor=1, pin_memory=True)
testIterator = iter(testDataLoader)

# get how many input channels there are
sample_X,sample_y = next(testIterator)
in_channels = sample_X.shape[1]
out_channels = sample_y.shape[1]